# System Aggregated State Probabilities Example

This example demonstrates how to compute system-wide aggregated state probabilities.

In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.ctmc.*
import jline.solvers.nc.*
import jline.solvers.mva.*

In [ ]:
// Set verbose level
GlobalConstants.setVerbose(VerboseLevel.STD)

// Create network for system aggregated probabilities
val model = Network("systemModel")

// Block 1: nodes
val node1 = Delay(model, "Delay")
val node2 = Queue(model, "Queue1", SchedStrategy.PS)
val node3 = Queue(model, "Queue2", SchedStrategy.FCFS)
val node4 = Queue(model, "Queue3", SchedStrategy.PS)
node3.setNumberOfServers(2)
node4.setNumberOfServers(1)

In [ ]:
// Block 2: classes
val jobclass1 = ClosedClass(model, "Class1", 3, node1, 0)
val jobclass2 = ClosedClass(model, "Class2", 2, node1, 0)

In [ ]:
// Set service times
node1.setService(jobclass1, Exp.fitMean(0.5))
node1.setService(jobclass2, Exp.fitMean(0.5))

node2.setService(jobclass1, Exp.fitMean(1.0))
node2.setService(jobclass2, Exp.fitMean(1.5))

node3.setService(jobclass1, Exp.fitMean(0.8))
node3.setService(jobclass2, Exp.fitMean(1.2))

node4.setService(jobclass1, Exp.fitMean(1.1))
node4.setService(jobclass2, Exp.fitMean(0.9))

In [ ]:
// Block 3: routing - more complex routing
val P = model.initRoutingMatrix()

// Class1 routing: Delay -> Queue1 -> Queue2 -> Queue3 -> Delay
P.set(jobclass1, jobclass1, node1, node2, 1.0)
P.set(jobclass1, jobclass1, node2, node3, 1.0)
P.set(jobclass1, jobclass1, node3, node4, 1.0)
P.set(jobclass1, jobclass1, node4, node1, 1.0)

// Class2 routing: Delay -> Queue2 -> Queue1 -> Queue3 -> Delay  
P.set(jobclass2, jobclass2, node1, node3, 1.0)
P.set(jobclass2, jobclass2, node3, node2, 1.0)
P.set(jobclass2, jobclass2, node2, node4, 1.0)
P.set(jobclass2, jobclass2, node4, node1, 1.0)

model.link(P)

In [ ]:
// Solve with CTMC for system aggregated probabilities
println("Solving with CTMC for system aggregated probabilities:")
try {
    val solverCtmc = CTMC(model)
    val avgTable = solverCtmc.avgTable
    println("CTMC Results:")
    avgTable.print()
    
    // Get system aggregated probabilities
    println("\nTesting system aggregated probabilities:")
    try {
        val sysProbs = solverCtmc.probSysAggr
        println("System aggregated probabilities computed successfully")
        println("Number of aggregated states: ${sysProbs.size}")
        
        // Print first few probabilities if available
        if (sysProbs.isNotEmpty()) {
            println("Sample aggregated probabilities:")
            for (i in 0 until minOf(5, sysProbs.size)) {
                println("  State $i: ${sysProbs[i]}")
            }
        }
    } catch (e: Exception) {
        println("System aggregated probabilities not available: ${e.message}")
    }
    
} catch (e: Exception) {
    println("CTMC solver error: ${e.message}")
}

In [ ]:
// Test with NC solver for comparison
println("\nTesting with NC solver:")
try {
    val solverNc = NC(model)
    val avgTableNc = solverNc.avgTable
    println("NC Results:")
    avgTableNc.print()
    
    // Test node-specific state probabilities
    node2.setState(intArrayOf(1, 0)) // State [1,0] at Queue1
    val nodeProb = solverNc.getProbAggr(node2)
    println("\nProbability of state [1,0] at Queue1: $nodeProb")
    
} catch (e: Exception) {
    println("NC solver error: ${e.message}")
}